In [1]:
import qubx  # type: ignore
from qubx.core.basics import DataType

%qubxd

%load_ext autoreload
%autoreload 2

# - - - - - - - - - -
from qubx.data.transformers import PandasFrame, TypedRecords, OHLCVSeries
from qubx.data.storage import RawData
from qubx.data.storages.csv import CsvStorage


⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2025, ver. 0.6.84
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


## Basic CSV storage tests

In [2]:
r = CsvStorage("~/devs/Qubx/tests/data/storages/csv")

print(r.get_exchanges())
print(r.get_market_types("BINANCE.UM"))

['BINANCE.UM']
['SWAP']


## Reader

In [27]:
bnc_swap = r["BINANCE.UM", "SWAP"]

In [28]:
bnc_swap.get_data_types("BTCUSDT")

['ohlc(1h)', quote]

In [29]:
bnc_swap.get_data_id("ohlc(1h)")

['ETHUSDT', 'BCHUSDT', 'BTCUSDT', 'LTCUSDT', 'AAVEUSDT']

In [6]:
print(bnc_swap.get_time_range("BTCUSDT", DataType.OHLC["1h"]))
print(bnc_swap.get_time_range("BTCUSDT", DataType.QUOTE))

(numpy.datetime64('2023-06-01T00:00:00'), numpy.datetime64('2023-08-01T00:00:00'))
(numpy.datetime64('2017-08-24T13:01:12'), numpy.datetime64('2017-08-24T13:01:59'))


In [19]:
ds = bnc_swap.read("BTCUSDT", DataType.OHLC['1h'], chunksize=0) # type: ignore

In [35]:
print(bnc_swap.read("BTCUSDT", DataType.QUOTE).raw[:10])

[[Timestamp('2017-08-24 13:01:12') 10.1 10.0 200 100]
 [Timestamp('2017-08-24 13:01:13') 10.1 10.01 20000 100]
 [Timestamp('2017-08-24 13:01:14') 10.13 10.03 200 100]
 [Timestamp('2017-08-24 13:01:15') 10.12 10.01 20000 100]
 [Timestamp('2017-08-24 13:01:16') 10.11 10.1 200 100]
 [Timestamp('2017-08-24 13:01:17') 10.19 10.16 200 100]
 [Timestamp('2017-08-24 13:01:18') 10.19 10.16 200 10000]
 [Timestamp('2017-08-24 13:01:19') 10.19 10.15 200 100]
 [Timestamp('2017-08-24 13:01:20') 10.18 10.15 20000 10000]
 [Timestamp('2017-08-24 13:01:21') 10.18 10.15 200 10000]]


In [32]:
print(bnc_swap.read("ETHUSDT", DataType.TRADE).raw[:10])

[[3920436870 3054.91 0.1 305.491 1713571200068 True]
 [3920436871 3054.91 0.144 439.90704 1713571200199 True]
 [3920436872 3054.91 0.328 1002.01048 1713571200199 True]
 [3920436873 3054.91 0.328 1002.01048 1713571200202 True]
 [3920436874 3054.91 0.037 113.03167 1713571200205 True]
 [3920436875 3054.92 0.007 21.38444 1713571200209 False]
 [3920436876 3054.91 0.328 1002.01048 1713571200225 True]
 [3920436877 3054.91 0.917 2801.35247 1713571200279 True]
 [3920436878 3054.92 0.027 82.48284 1713571200297 False]
 [3920436879 3054.92 0.174 531.55608 1713571200297 False]]


In [33]:
print(bnc_swap.read("ETHUSDT", DataType.OHLC["1Min"]).raw[:10])

ValueError: Can't find any csv data for 'ETHUSDT' of 'ohlc(1Min)' in /home/quant0/devs/Qubx/tests/data/storages/csv/BINANCE.UM/SWAP !

## Conversion

In [44]:
r = CsvStorage("~/devs/Qubx/tests/data/storages/csv")
bnc_swap = r["BINANCE.UM", "SWAP"]

raw1 = bnc_swap.read("BTCUSDT", DataType.QUOTE, None, None)
raw2 = bnc_swap.read("BTCUSDT", DataType.OHLC["1h"], None, None)
raw3 = bnc_swap.read("ETHUSDT", DataType.OHLC["1h"], None, None)
raw4 = bnc_swap.read("ETHUSDT", DataType.TRADE, None, None)
raw5 = bnc_swap.read("BTCUSDT", DataType.QUOTE, None, None)

In [45]:
raw4.transform(PandasFrame())

,id,price,qty,quote_qty,is_buyer_maker
time,,,,,
1970-01-01 00:28:33.571200068,3920436870,3054.91,0.1,305.491,True
1970-01-01 00:28:33.571200199,3920436871,3054.91,0.144,439.90704,True
1970-01-01 00:28:33.571200199,3920436872,3054.91,0.328,1002.01048,True
1970-01-01 00:28:33.571200202,3920436873,3054.91,0.328,1002.01048,True
1970-01-01 00:28:33.571200205,3920436874,3054.91,0.037,113.03167,True
...,...,...,...,...,...
1970-01-01 00:28:33.571229272,3920438865,3056.52,0.01,30.5652,True
1970-01-01 00:28:33.571229272,3920438866,3056.52,0.005,15.2826,True
1970-01-01 00:28:33.571229315,3920438867,3056.52,4.674,14286.17448,False


In [46]:
raw5.get_time_interval()

(Timestamp('2017-08-24 13:01:12'), Timestamp('2017-08-24 13:01:59'))

In [47]:
raw4.transform(TypedRecords())

[[1970-01-01T00:28:33.571200068]	3054.91000 (0.10) buy 3920436870,
 [1970-01-01T00:28:33.571200199]	3054.91000 (0.14) buy 3920436871,
 [1970-01-01T00:28:33.571200199]	3054.91000 (0.33) buy 3920436872,
 [1970-01-01T00:28:33.571200202]	3054.91000 (0.33) buy 3920436873,
 [1970-01-01T00:28:33.571200205]	3054.91000 (0.04) buy 3920436874,
 [1970-01-01T00:28:33.571200209]	3054.92000 (0.01) ??? 3920436875,
 [1970-01-01T00:28:33.571200225]	3054.91000 (0.33) buy 3920436876,
 [1970-01-01T00:28:33.571200279]	3054.91000 (0.92) buy 3920436877,
 [1970-01-01T00:28:33.571200297]	3054.92000 (0.03) ??? 3920436878,
 [1970-01-01T00:28:33.571200297]	3054.92000 (0.17) ??? 3920436879,
 [1970-01-01T00:28:33.571200297]	3054.92000 (0.03) ??? 3920436880,
 [1970-01-01T00:28:33.571200297]	3054.92000 (0.03) ??? 3920436881,
 [1970-01-01T00:28:33.571200297]	3054.92000 (0.09) ??? 3920436882,
 [1970-01-01T00:28:33.571200325]	3054.92000 (0.02) ??? 3920436883,
 [1970-01-01T00:28:33.571200335]	3054.92000 (0.01) ??? 3920436

In [48]:
raw5.transform(TypedRecords())

[[2017-08-24T13:01:12.000000000]	10.00000 (100.00) | 10.10000 (200.00),
 [2017-08-24T13:01:13.000000000]	10.01000 (100.00) | 10.10000 (20000.00),
 [2017-08-24T13:01:14.000000000]	10.03000 (100.00) | 10.13000 (200.00),
 [2017-08-24T13:01:15.000000000]	10.01000 (100.00) | 10.12000 (20000.00),
 [2017-08-24T13:01:16.000000000]	10.10000 (100.00) | 10.11000 (200.00),
 [2017-08-24T13:01:17.000000000]	10.16000 (100.00) | 10.19000 (200.00),
 [2017-08-24T13:01:18.000000000]	10.16000 (10000.00) | 10.19000 (200.00),
 [2017-08-24T13:01:19.000000000]	10.15000 (100.00) | 10.19000 (200.00),
 [2017-08-24T13:01:20.000000000]	10.15000 (10000.00) | 10.18000 (20000.00),
 [2017-08-24T13:01:21.000000000]	10.15000 (10000.00) | 10.18000 (200.00),
 [2017-08-24T13:01:22.000000000]	10.04000 (100.00) | 10.15000 (20000.00),
 [2017-08-24T13:01:23.000000000]	10.06000 (100.00) | 10.19000 (200.00),
 [2017-08-24T13:01:24.000000000]	10.04000 (100.00) | 10.16000 (200.00),
 [2017-08-24T13:01:25.000000000]	10.03000 (10000.0

In [49]:
raw2.transform(TypedRecords())

[[2023-06-01T00:00:00.000000000] {o:27201.100000 | h:27347.900000 | l:27008.100000 | c:27059.800000 | v:33639.568000},
 [2023-06-01T01:00:00.000000000] {o:27059.800000 | h:27155.700000 | l:27045.700000 | c:27065.500000 | v:17636.072000},
 [2023-06-01T02:00:00.000000000] {o:27065.500000 | h:27106.500000 | l:26621.300000 | c:26705.100000 | v:42682.676000},
 [2023-06-01T03:00:00.000000000] {o:26705.000000 | h:26829.000000 | l:26600.000000 | c:26768.600000 | v:27103.262000},
 [2023-06-01T04:00:00.000000000] {o:26768.600000 | h:26813.600000 | l:26742.000000 | c:26797.200000 | v:9880.983000},
 [2023-06-01T05:00:00.000000000] {o:26797.100000 | h:26858.700000 | l:26771.800000 | c:26842.900000 | v:8112.849000},
 [2023-06-01T06:00:00.000000000] {o:26843.000000 | h:26876.600000 | l:26762.000000 | c:26777.600000 | v:11210.559000},
 [2023-06-01T07:00:00.000000000] {o:26777.600000 | h:26856.000000 | l:26753.100000 | c:26790.200000 | v:10103.296000},
 [2023-06-01T08:00:00.000000000] {o:26790.100000 |

In [50]:
raw2.transform(PandasFrame(True))

,,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume
timestamp,symbol,,,,,,,,,
2023-06-01 00:00:00,BTCUSDT,27201.1,27347.9,27008.1,27059.8,33639.568,914441790.0,256082,15501.039,421503008.0
2023-06-01 01:00:00,BTCUSDT,27059.8,27155.7,27045.7,27065.5,17636.072,478001888.0,141280,8794.62,238385216.0
2023-06-01 02:00:00,BTCUSDT,27065.5,27106.5,26621.3,26705.1,42682.676,1144205950.0,268982,17221.14,461549856.0
2023-06-01 03:00:00,BTCUSDT,26705.0,26829.0,26600.0,26768.6,27103.262,724444350.0,219415,14888.465,398015552.0
2023-06-01 04:00:00,BTCUSDT,26768.6,26813.6,26742.0,26797.2,9880.983,264626432.0,94780,5605.699,150137440.0
...,...,...,...,...,...,...,...,...,...,...
2023-07-31 20:00:00,BTCUSDT,29146.4,29249.0,29102.3,29194.5,13962.28,407463040.0,127312,7628.219,222628592.0
2023-07-31 21:00:00,BTCUSDT,29194.5,29285.3,29144.8,29230.6,10977.811,320717216.0,86779,5551.999,162209840.0
2023-07-31 22:00:00,BTCUSDT,29230.6,29232.1,29186.0,29199.6,4243.307,123939200.0,43890,1955.245,57107704.0


In [58]:
raw2.transform(OHLCVSeries())

                        open     high      low    close     volume  \
timestamp                                                            
2023-06-01 00:00:00  27201.1  27347.9  27008.1  27059.8  33639.568   
2023-06-01 01:00:00  27059.8  27155.7  27045.7  27065.5  17636.072   
2023-06-01 02:00:00  27065.5  27106.5  26621.3  26705.1  42682.676   
2023-06-01 03:00:00  26705.0  26829.0  26600.0  26768.6  27103.262   
2023-06-01 04:00:00  26768.6  26813.6  26742.0  26797.2   9880.983   
...                      ...      ...      ...      ...        ...   
2023-07-31 20:00:00  29146.4  29249.0  29102.3  29194.5  13962.280   
2023-07-31 21:00:00  29194.5  29285.3  29144.8  29230.6  10977.811   
2023-07-31 22:00:00  29230.6  29232.1  29186.0  29199.6   4243.307   
2023-07-31 23:00:00  29199.5  29229.8  29173.4  29220.8   3765.949   
2023-08-01 00:00:00  29220.8  29277.0  29200.9  29272.3   4934.267   

                     bought_volume  volume_quote  bought_volume_quote  \
timestamp       

In [ ]:
srows(raw2.transform(PandasFrame(True)), raw3.transform(PandasFrame(True)))

In [39]:
t0 = np.datetime64("2020-01-01", "ns").item()
dt = pd.Timedelta("1h").asm8.item()

r1 = RawData(
    "TEST1", ["time", "price", "size"],
    DataType.NONE,
    [ [(t0 + k * dt), 100 + k, k*0.5] for k in range(24) ]
)

In [106]:
r1.transform(TypedRecords())#[0].time

[[2020-01-01T00:00:00.000000000]	 {'price': 100, 'size': 0.0},
 [2020-01-01T01:00:00.000000000]	 {'price': 101, 'size': 0.5},
 [2020-01-01T02:00:00.000000000]	 {'price': 102, 'size': 1.0},
 [2020-01-01T03:00:00.000000000]	 {'price': 103, 'size': 1.5},
 [2020-01-01T04:00:00.000000000]	 {'price': 104, 'size': 2.0},
 [2020-01-01T05:00:00.000000000]	 {'price': 105, 'size': 2.5},
 [2020-01-01T06:00:00.000000000]	 {'price': 106, 'size': 3.0},
 [2020-01-01T07:00:00.000000000]	 {'price': 107, 'size': 3.5},
 [2020-01-01T08:00:00.000000000]	 {'price': 108, 'size': 4.0},
 [2020-01-01T09:00:00.000000000]	 {'price': 109, 'size': 4.5},
 [2020-01-01T10:00:00.000000000]	 {'price': 110, 'size': 5.0},
 [2020-01-01T11:00:00.000000000]	 {'price': 111, 'size': 5.5},
 [2020-01-01T12:00:00.000000000]	 {'price': 112, 'size': 6.0},
 [2020-01-01T13:00:00.000000000]	 {'price': 113, 'size': 6.5},
 [2020-01-01T14:00:00.000000000]	 {'price': 114, 'size': 7.0},
 [2020-01-01T15:00:00.000000000]	 {'price': 115, 'size'

In [107]:

r4 = RawData(
    "TEST1",
    ["timestamp", "symbol", "top_bid", "top_ask", "tick_size", "bid_100", "ask_1", "bid_1", "ask_100"],
    DataType.ORDERBOOK,
    [[t0 + k * dt, "BTCUSDT", 90_000.0 + k, 90_000.1 + k, 0.05, 100.0, 20.0, 10.0, 200.0] for k in range(24)],
)
r4.transform(TypedRecords())

[[2020-01-01T00:00:00.000000000] 90000.0 (10.0) | 90000.1 (20.0),
 [2020-01-01T01:00:00.000000000] 90001.0 (10.0) | 90001.1 (20.0),
 [2020-01-01T02:00:00.000000000] 90002.0 (10.0) | 90002.1 (20.0),
 [2020-01-01T03:00:00.000000000] 90003.0 (10.0) | 90003.1 (20.0),
 [2020-01-01T04:00:00.000000000] 90004.0 (10.0) | 90004.1 (20.0),
 [2020-01-01T05:00:00.000000000] 90005.0 (10.0) | 90005.1 (20.0),
 [2020-01-01T06:00:00.000000000] 90006.0 (10.0) | 90006.1 (20.0),
 [2020-01-01T07:00:00.000000000] 90007.0 (10.0) | 90007.1 (20.0),
 [2020-01-01T08:00:00.000000000] 90008.0 (10.0) | 90008.1 (20.0),
 [2020-01-01T09:00:00.000000000] 90009.0 (10.0) | 90009.1 (20.0),
 [2020-01-01T10:00:00.000000000] 90010.0 (10.0) | 90010.1 (20.0),
 [2020-01-01T11:00:00.000000000] 90011.0 (10.0) | 90011.1 (20.0),
 [2020-01-01T12:00:00.000000000] 90012.0 (10.0) | 90012.1 (20.0),
 [2020-01-01T13:00:00.000000000] 90013.0 (10.0) | 90013.1 (20.0),
 [2020-01-01T14:00:00.000000000] 90014.0 (10.0) | 90014.1 (20.0),
 [2020-01-